# Chapter 4, Example 4.2

The excercise deals with a 2D truss; to make things more general, the code is written to handle 3D elements as well. This truss can be computed as a 2D one, for which:
- the node coordinates must be in a 2D plane, i.e. the z-coordinates must be zero,
- set the supports to match the dimensionality.

Set the dimensionality for **this** problem in the Geometry section.

In [1]:
# imports
import numpy as np
from source.node import Node
from source.truss.truss import TrussModel
np.set_printoptions(precision=3, suppress=False)

### Geometry

Nodes are defined by their coordinates in 2D or 3D. ID is automatically assigned based on the order of creation, the first ID is 0.

In [2]:
dimensionality = 3 # Set to 3 for a 3D truss, 2 for a 2D truss

if dimensionality == 3:
    # 3D nodes
    n1 = Node(0, 0, 0)
    n2 = Node(1, 0, 0)
    n3 = Node(0, 1, 0)
elif dimensionality == 2:
    # 2D nodes
    n1 = Node(0, 0)
    n2 = Node(1, 0)
    n3 = Node(0, 1)
else:
    raise ValueError("Unsupported dimensionality. Use 2 or 3.")

A = 0.1  # Cross-sectional area
E = 7e10  # Young's modulus
ro = 1  # Density, not used but required.

### Truss Model

The model handles all aspects of the truss: nodes, topology, supports, loads.

- Nodes: simply listed as a tuple of Node objects defined earlier.
- Elements: nodes are provided by ID, the rest is area, material properties.
- Supports: defined by node ID and direction, where 'x', 'y', 'z' are the directions. **The correctness of the support input is not checked; if you encounter a singular stiffness matrix, check the inputs first.**

In [3]:
# model

supports_=((0, 'x'), (0, 'y'), (2, 'x'),)
if dimensionality == 3:
    supports_ += ((0, 'z'), (1, 'z'), (2, 'z'))

model = TrussModel(
    nodes_=(n1, n2, n3),
    elements_=((n1.ID, n2.ID, A, E, ro),
               (n1.ID, n3.ID, A, E, ro),
               (n2.ID, n3.ID, A, E, ro),),
    supports_=supports_,
)

# loads
_F = np.zeros(model.ND * len(model.nodes))  # Global force vector, initialized to zero
_F[model.ND + 1] = -1000  # Apply a force of -1000 N in the y-direction at node 2 (index 1)

print(f'The model has {model.ND} degrees of freedom per node.')

The model has 3 degrees of freedom per node.


### Nodes and their properties

Internally the nodes are stored in a dictionary with the node ID as the key. IDs start at 0.

In [4]:
for _, node in model.nodes.items():
    print(f"Node {node.ID}: coords:{node.coords}")

Node 0: coords:[0 0 0]
Node 1: coords:[1 0 0]
Node 2: coords:[0 1 0]


### The elements and their properties

Internally the elements are stored in a dictionary with the element ID as the key. IDs start at 0.

In [5]:
for _, element in model.elements.items():
    print(f"Element {element.ID}: Ae={element.A}, le={element.length:.2f}, E={element.E:g}")

for _, element in model.elements.items():
    print(f"Element {element.ID}: node i={element.i.coords}, node j={element.j.coords}, direction={element.direction_vector}")


Element 0: Ae=0.1, le=1.00, E=7e+10
Element 1: Ae=0.1, le=1.00, E=7e+10
Element 2: Ae=0.1, le=1.41, E=7e+10
Element 0: node i=[0 0 0], node j=[1 0 0], direction=[1 0 0]
Element 1: node i=[0 0 0], node j=[0 1 0], direction=[0 1 0]
Element 2: node i=[1 0 0], node j=[0 1 0], direction=[-1  1  0]


### Element stiffness matrices



In [6]:
for _, element in model.elements.items():
    print()
    print(f"Element {element.ID} stiffness matrix in the element local coordinate system:\n{element.ke}\n")
    print(f"Element {element.ID} stiffness matrix in the global coordinate system:\n{element.Ke}\n")


Element 0 stiffness matrix in the element local coordinate system:
[[ 7.e+09 -7.e+09]
 [-7.e+09  7.e+09]]

Element 0 stiffness matrix in the global coordinate system:
[[ 7.e+09  0.e+00  0.e+00 -7.e+09  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [-7.e+09  0.e+00  0.e+00  7.e+09  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]]


Element 1 stiffness matrix in the element local coordinate system:
[[ 7.e+09 -7.e+09]
 [-7.e+09  7.e+09]]

Element 1 stiffness matrix in the global coordinate system:
[[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  7.e+09  0.e+00  0.e+00 -7.e+09  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00 -7.e+09  0.e+00  0.e+00  7.e+09  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]]


Element 2 stiffness matrix in the element local

#### The global stiffness matrix

First, without the supports, as assembled from the element stiffness matrices.

To make matrix handling easier, the supports are taken into account by adding a penalty instead of eliminating the rows and columns. This is done by adding a large value to the diagonal of the stiffness matrix for the supported degrees of freedom and setting the respective elements in the load vector to zero.

In [7]:
np.set_printoptions(precision=2, suppress=False, threshold=120)
print('Global stiffness matrix without supports:')
print(model.K)
print()
print('Global stiffness matrix with supports:')
_K, _F = model.apply_boundary_conditions(F=_F)
print(_K)
print()
print('Global force vector with supports:')
print(_F)

Global stiffness matrix without supports:
[[ 7.00e+09  0.00e+00  0.00e+00 -7.00e+09  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  0.00e+00]
 [ 0.00e+00  7.00e+09  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
  -7.00e+09  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  0.00e+00]
 [-7.00e+09  0.00e+00  0.00e+00  9.47e+09 -2.47e+09  0.00e+00 -2.47e+09
   2.47e+09  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00 -2.47e+09  2.47e+09  0.00e+00  2.47e+09
  -2.47e+09  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00 -2.47e+09  2.47e+09  0.00e+00  2.47e+09
  -2.47e+09  0.00e+00]
 [ 0.00e+00 -7.00e+09  0.00e+00  2.47e+09 -2.47e+09  0.00e+00 -2.47e+09
   9.47e+09  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  0.00e+00]]

Global stiffness matrix with supports:
[[ 1.00e+20  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 

### Solving the system

In [8]:
U = np.linalg.solve(_K, _F)
np.set_printoptions(precision=2, suppress=False, threshold=2)

print()
print("Global displacements:")
for i, node in model.nodes.items():
    if model.ND == 2:
        print(f"Node {i}: u={U[i * model.ND]}, v={U[i * model.ND + 1]}")
    else:
        # For 3D, we have an additional 'w' coordinate
        print(f"Node {i}: u={U[i * model.ND]}, v={U[i * model.ND + 1]}, w={U[i * model.ND + 2]}")

# Calculate the member force using the local stiffness matrix
print()
print('Member forces:')

for id_, (force, direction) in enumerate(model.member_forces(U)):
    print(f"Element {id_}: force={force:.2f} N, ({direction})")

# Calculate the reaction forces
print()
print('Vector of reaction forces for the global DOFs:')
for dof, value in enumerate(model.reaction_forces(U, _F)):
    print(f"DOF {dof}: {value:.2f} N")



Global displacements:
Node 0: u=0.0, v=0.0, w=0.0
Node 1: u=-1.4285714285714285e-07, v=-6.897753035351702e-07, w=0.0
Node 2: u=0.0, v=-1.4285714285714285e-07, w=0.0

Member forces:
Element 0: force=-1000.00 N, (compression)
Element 1: force=-1000.00 N, (compression)
Element 2: force=1414.21 N, (tension)

Vector of reaction forces for the global DOFs:
DOF 0: 1000.00 N
DOF 1: 1000.00 N
DOF 2: 0.00 N
DOF 3: 0.00 N
DOF 4: -0.00 N
DOF 5: 0.00 N
DOF 6: -1000.00 N
DOF 7: 0.00 N
DOF 8: 0.00 N


### Plotting the original and the deformed shape

In [9]:
%matplotlib notebook
model.plot_truss(U)

<IPython.core.display.Javascript object>